# Constantes y Funciones Auxiliares

In [1]:
# Semilla
SEED = 333

# Preparamos el lematizado
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

# Lematizar un string
import re
def stemming_tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [porter_stemmer.stem(word) for word in words]
    return words

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

## Columnas de guardado para los algortimos 
COLUMNS = ['mean_fit_time','std_fit_time','mean_test_neg_log_loss','std_test_neg_log_loss','rank_test_neg_log_loss',
           'mean_test_accuracy','rank_test_accuracy',
           'mean_test_f1_macro','rank_test_f1_macro',
           'mean_test_roc_auc_ovr','rank_test_roc_auc_ovr']

# Funcion de guardado de resultados que es un subconjunto de cv_results. 
# Guarda los resultados de los parametros del algoritmo y las metricas que le pasamos como parametro.
def save_results(rs,params_to_evaluate,columns=COLUMNS):
    aux = pd.DataFrame(rs.cv_results_)
    gs_res = pd.DataFrame()
    for col in params_to_evaluate:
        gs_res[col] = aux[col]
    for col in columns:
        gs_res[col] = aux[col]
    return gs_res

# Carga

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

train_variants_df = pd.read_csv(r"C:\Users\Junio\Libretas\data-c\training_variants", engine='python')
train_txt_df = pd.read_csv(r"C:\Users\Junio\Libretas\data-c/training_text", sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
train_txt_df['Class'] = train_variants_df['Class']
train_txt_df.sample(10,random_state=SEED)

,ID,Text,Class
3198,3198,Inactivation of Ras GTPase activating proteins...,1
2161,2161,The PTEN (phosphatase and tensin homolog) phos...,4
1083,1083,"EZH2, the catalytic subunit of the PRC2 comple...",9
113,113,Endometrial cancer is the most common gynecolo...,4
885,885,Purpose: Platelet-derived growth factor recept...,7
3210,3210,Hereditary predisposition to retinoblastoma is...,1
153,153,Many studies have reported the EGFR mutations ...,2
2312,2312,A systematic characterization of the genetic a...,7
1107,1107,"Multiple endocrine neoplasia type 1 (MEN1, OM...",4
1648,1648,"In acute myeloid leukemia (AML), two clusters ...",7


# Preparacion del Dataframe

In [3]:
# Inicializamos el dataframe que vamos a utilizar
W = pd.DataFrame()

# Añadimos una columna que nos indica el tamaño del texto de cada instancia
W['Text_count']  = train_txt_df["Text"].apply(lambda x: len(str(x).split()))

# Copiamos la clase y el texto
W['Class'] = train_txt_df['Class'].copy()
W['Text'] = train_txt_df["Text"].copy()

# Nos quedamos con las instancias que no tengan el texto nulo
W = W[W['Text_count']!=1]

# Mostramos el dataframe
W.sample(10,random_state=SEED)

,Text_count,Class,Text
3198,3343,1,Inactivation of Ras GTPase activating proteins...
2084,3005,7,MYD88 L265P is a somatic mutation that has bee...
3185,11198,6,We describe the case of a patient presenting w...
113,19533,4,Endometrial cancer is the most common gynecolo...
3133,15135,7,Transforming mutations in NRAS and KRAS are th...
1110,14851,1,Abstract Fanconi anemia is characterized by c...
153,3837,2,Many studies have reported the EGFR mutations ...
1242,65740,7,The development of array comparative genomic h...
2768,6895,7,Over 30 mutations of the B-RAF gene associated...
1045,10154,1,Tuberous sclerosis (TSC) is an autosomal domin...


# Preparando la Clasificacion 

In [4]:
# Separacion training/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(W['Text'], W['Class'], test_size=0.2, random_state=SEED)

In [5]:
# Preparando stop_words
from wordcloud import STOPWORDS

# Capturando las palabras que no se encuentran en STOPWORDS por ser una contraccion de la palabra original (salida de stemming_tokenizer)
contract_words = {'abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 
              'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 
              'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 
              'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 
              'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 
              'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 
              'someth', 'sometim', 'somewher', 'themselv', 'thenc', 
              'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 
              'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 
              'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'}

l2 = {'anywh', 'aren', 'becau', 'couldn', 'd', 'didn', 'doe', 
      'doesn', 'don', 'el', 'elsewh', 'everywh', 'hadn', 'hasn', 
      'haven', 'ind', 'isn', 'let', 'll', 'm', 'mustn', 
      'otherwi', 'plea', 're', 's', 'shan', 'shouldn', 
      'somewh', 't', 've', 'wasn', 'weren', 'won', 'wouldn'}

custom_words = {"fig", "figure", "et", "al", "al.", "also",
                "data", "analyze", "study", "table", "using",
                "method", "result", "conclusion", "author", 
                "find", "found", "show", "casita","non","name","image",
                'analyz', 'conclus', 'figur','conclu', 'imag', 'studi', 'tabl', 'use'}

# Unimos ambas listas
stop_words = STOPWORDS.union(contract_words).union(l2)

#from sklearn.pipeline import Pipeline
from imblearn.combine import SMOTETomek
# Para crear pipelines de manera sencilla
from imblearn.pipeline import make_pipeline as make_pipeline
def create_pipeline(clf,ngram_range=(1,1)):
    tf_idf = TfidfVectorizer(analyzer="word", tokenizer=stemming_tokenizer,stop_words= stop_words,ngram_range=ngram_range)
    aumento = SMOTETomek(random_state=SEED)
    estimator = clf
    return make_pipeline(tf_idf,aumento,estimator)


# Validacion Cruzada Stratificada(n_splits=3):
from sklearn.model_selection import StratifiedKFold
CV = StratifiedKFold(n_splits=3, random_state=SEED, shuffle=True)

from sklearn.model_selection import RandomizedSearchCV
def create_rscv(pipeline,params,n_iterations = 10,scoring = ["neg_log_loss","accuracy","f1_macro","roc_auc_ovr"],cv = CV):
    return RandomizedSearchCV(
            pipeline,
            params,
            n_iter = n_iterations,
            verbose = 1,
            random_state = SEED,
            cv = cv,
            n_jobs = -1,
            scoring = scoring,
            refit = "neg_log_loss" 
            )
# Importamos la metrica principal de evaluacion
from sklearn import metrics

# Dataframe de guardado del test
df_results = pd.DataFrame(columns = ["clf","log_loss","accuracy","f1-macro","ROC"])

# Funcion de guardado del resultado del test
def add_res(clf,name,X_test = X_test):   
    # Guardamos las predicciones
    y_predict_proba = clf.predict_proba(X_test)
    y_pred = clf.predict(X_test)
    
    # Guardamos los resultados de las distintas metricas
    log_loss = metrics.log_loss(y_test,y_predict_proba)
    acc = metrics.accuracy_score(y_test,y_pred)
    f1 = metrics.f1_score(y_test, y_pred, average='macro')
    roc = metrics.roc_auc_score(y_test,y_predict_proba,multi_class='ovr')
    
    # Actualizamos el dataframe
    df_results.loc[len(df_results)]=[name,log_loss,acc,f1,roc]

## Clasificacion

## Naive Bayes Multinomial

In [6]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
parameters = {
    'tfidfvectorizer__ngram_range': ((1, 1),(2, 2),(1,2)),
    'tfidfvectorizer__use_idf': (True, False),
    'tfidfvectorizer__max_features': (1000,2000,3000,4000,5000,6000,7000,8000,9000,None),
    'tfidfvectorizer__norm': ('l1', 'l2')
 }
pipeline = create_pipeline(clf)
rs_NB_M = create_rscv(pipeline,parameters)

In [ ]:
rs_NB_M.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [8]:
params_to_evaluate = ["param_tfidfvectorizer__max_features","param_tfidfvectorizer__ngram_range","param_tfidfvectorizer__use_idf",
                      "param_tfidfvectorizer__norm"]
NB_M_res = save_results(rs_NB_M,params_to_evaluate)
NB_M_res.sort_values(by='mean_test_neg_log_loss',ascending=False).head(5)

,param_tfidfvectorizer__max_features,param_tfidfvectorizer__ngram_range,param_tfidfvectorizer__use_idf,param_tfidfvectorizer__norm,mean_fit_time,std_fit_time,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,mean_test_accuracy,rank_test_accuracy,mean_test_f1_macro,rank_test_f1_macro,mean_test_roc_auc_ovr,rank_test_roc_auc_ovr
8,5000,"(1, 1)",True,l2,1039.434027,18.850769,-1.391355,0.060023,1,0.539593,1,0.472142,1,0.857517,1
7,1000,"(2, 2)",True,l2,1059.400635,10.737122,-1.444794,0.057871,2,0.495098,2,0.413895,2,0.846698,7
0,3000,"(2, 2)",True,l1,997.131005,5.628674,-2.108364,0.001107,3,0.413650,5,0.366663,5,0.849269,5
4,5000,"(1, 1)",True,l1,1007.245070,24.140710,-2.133374,0.001456,4,0.417798,4,0.375975,4,0.848803,6
9,6000,"(2, 2)",True,l1,788.239446,147.878577,-2.137689,0.002027,5,0.389140,7,0.360676,6,0.851932,3


In [9]:
# Exportamos los resultados
NB_M_res.to_csv('NB_M.csv')

# Testing
add_res(rs_NB_M,'NB_M')

## KNN

In [10]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
parameters = {
    'tfidfvectorizer__ngram_range': ((1, 1),(2, 2),(1,2)),
    'tfidfvectorizer__use_idf': (True, False),
    'tfidfvectorizer__max_features': (1000,2000,3000,4000,5000,6000,7000,8000,9000,None),
    'tfidfvectorizer__norm': ('l1', 'l2'),
    'kneighborsclassifier__n_neighbors': tuple(range(1,60,2))
 }
pipeline5 = create_pipeline(knn)
rs_KNN = create_rscv(pipeline5,parameters)

In [11]:
rs_KNN.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 234.2min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=333, shuffle=True),
                   estimator=Pipeline(steps=[('tfidfvectorizer',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'abov',
                                                                          'above',
                                                                          'after',
                                                                          'afterward',
                                                                          'again',
                                                                          'against',
                                                                          'all',
                                                                          'alon',
                        

In [12]:
params_to_evaluate = ["param_tfidfvectorizer__max_features","param_tfidfvectorizer__ngram_range","param_tfidfvectorizer__use_idf",
                      "param_tfidfvectorizer__norm","param_kneighborsclassifier__n_neighbors"]
KNN_res = save_results(rs_KNN,params_to_evaluate)
KNN_res.sort_values(by='mean_test_neg_log_loss',ascending=False).head(5)

,param_tfidfvectorizer__max_features,param_tfidfvectorizer__ngram_range,param_tfidfvectorizer__use_idf,param_tfidfvectorizer__norm,param_kneighborsclassifier__n_neighbors,mean_fit_time,std_fit_time,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,mean_test_accuracy,rank_test_accuracy,mean_test_f1_macro,rank_test_f1_macro,mean_test_roc_auc_ovr,rank_test_roc_auc_ovr
7,4000,"(1, 1)",True,l1,59,926.378875,6.178383,-2.612732,0.301800,1,0.328054,8,0.334451,6,0.816880,6
8,5000,"(1, 2)",False,l2,59,972.862212,7.849921,-3.072612,0.515209,2,0.290724,10,0.296639,9,0.825898,4
4,7000,"(2, 2)",True,l2,57,927.163444,6.515623,-3.717019,0.360032,3,0.297134,9,0.292893,10,0.814931,8
3,5000,"(1, 1)",False,l2,33,925.429747,5.687038,-3.826130,0.487326,4,0.353318,7,0.331658,7,0.829770,3
0,1000,"(2, 2)",True,l1,31,899.261407,10.545363,-3.846075,0.094189,5,0.366516,6,0.327754,8,0.815456,7


In [13]:
# Exportamos los resultados
KNN_res.to_csv('KNN.csv')

# Testing
add_res(rs_KNN,'KNN')

## Random Forest

In [14]:
from sklearn.ensemble import RandomForestClassifier
clf3 = RandomForestClassifier(random_state=SEED)
parameters = {
    'tfidfvectorizer__ngram_range': ((1, 1),(2, 2),(1,2)),
    'tfidfvectorizer__use_idf': (True, False),
    'tfidfvectorizer__max_features': (1000,2000,3000,4000,5000,6000,7000,8000,9000,None),
    'tfidfvectorizer__norm': ('l1', 'l2'),
    'randomforestclassifier__criterion':('giny','entropy')
 }
pipeline3 = create_pipeline(clf3)
rs_RF = create_rscv(pipeline3,parameters)

In [15]:
rs_RF.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 182.7min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=333, shuffle=True),
                   estimator=Pipeline(steps=[('tfidfvectorizer',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'abov',
                                                                          'above',
                                                                          'after',
                                                                          'afterward',
                                                                          'again',
                                                                          'against',
                                                                          'all',
                                                                          'alon',
                        

In [17]:
params_to_evaluate = ["param_tfidfvectorizer__max_features","param_tfidfvectorizer__ngram_range","param_tfidfvectorizer__use_idf",
                      "param_tfidfvectorizer__norm","param_randomforestclassifier__criterion"]
RF_res = save_results(rs_RF,params_to_evaluate)
RF_res.sort_values(by='mean_test_neg_log_loss',ascending=False).head(5)

,param_tfidfvectorizer__max_features,param_tfidfvectorizer__ngram_range,param_tfidfvectorizer__use_idf,param_tfidfvectorizer__norm,param_randomforestclassifier__criterion,mean_fit_time,std_fit_time,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,mean_test_accuracy,rank_test_accuracy,mean_test_f1_macro,rank_test_f1_macro,mean_test_roc_auc_ovr,rank_test_roc_auc_ovr
6,5000,"(1, 1)",True,l2,entropy,1061.978851,12.235525,-1.813907,0.100854,1,0.620287,1,0.538971,2,0.870337,1
3,9000,"(1, 1)",False,l1,entropy,1099.787059,13.298077,-1.856291,0.057779,2,0.610106,5,0.526712,5,0.866933,2
1,2000,"(1, 1)",False,l2,entropy,1024.090189,6.346614,-1.866947,0.145397,3,0.614630,4,0.537213,3,0.859295,3
9,1000,"(1, 2)",True,l2,entropy,876.802809,96.948109,-1.948730,0.029698,4,0.618401,2,0.542822,1,0.858182,4
8,4000,"(1, 1)",True,l1,entropy,1063.469198,7.682120,-1.979492,0.106470,5,0.617647,3,0.533798,4,0.854093,5


In [18]:
# Exportamos los resultados
RF_res.to_csv('RF.csv')

# Testing
add_res(rs_RF,'RF')

## SVC

In [19]:
from sklearn.svm import SVC
clf4 = SVC(probability=True)
parameters = {
    'tfidfvectorizer__ngram_range': ((1, 1),(2, 2),(1,2)),
    'tfidfvectorizer__use_idf': (True, False),
    'tfidfvectorizer__max_features': (1000,2000,3000,4000,5000,6000,7000,8000,9000,None),
    'tfidfvectorizer__norm': ('l1', 'l2'),
    'svc__kernel':('linear','rbf','sigmoid','poly')
 }
pipeline4 = create_pipeline(clf4)
rs_SVC = create_rscv(pipeline4,parameters)

In [21]:
rs_SVC.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 573.2min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=333, shuffle=True),
                   estimator=Pipeline(steps=[('tfidfvectorizer',
                                              TfidfVectorizer(stop_words={'a',
                                                                          'about',
                                                                          'abov',
                                                                          'above',
                                                                          'after',
                                                                          'afterward',
                                                                          'again',
                                                                          'against',
                                                                          'all',
                                                                          'alon',
                        

In [22]:
params_to_evaluate = ["param_tfidfvectorizer__max_features","param_tfidfvectorizer__use_idf","param_tfidfvectorizer__ngram_range","param_tfidfvectorizer__norm","param_svc__kernel"]
SVC_res = save_results(rs_SVC,params_to_evaluate)
SVC_res.sort_values(by='mean_test_neg_log_loss',ascending=False).head(5)

,param_tfidfvectorizer__max_features,param_tfidfvectorizer__use_idf,param_tfidfvectorizer__ngram_range,param_tfidfvectorizer__norm,param_svc__kernel,mean_fit_time,std_fit_time,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,mean_test_accuracy,rank_test_accuracy,mean_test_f1_macro,rank_test_f1_macro,mean_test_roc_auc_ovr,rank_test_roc_auc_ovr
3,5000,True,"(2, 2)",l2,rbf,1594.243199,8.100563,-1.144969,0.032005,1,0.624811,1,0.543988,1,0.883414,3
1,7000,True,"(1, 1)",l2,rbf,2068.816854,4.556349,-1.150726,0.021940,2,0.620664,2,0.535051,2,0.891259,1
7,5000,False,"(1, 1)",l2,linear,1836.320048,7.236467,-1.174417,0.032429,3,0.594268,4,0.528394,3,0.878716,5
8,7000,True,"(1, 1)",l2,sigmoid,1841.263618,15.566603,-1.194646,0.009901,4,0.578054,6,0.501365,8,0.883805,2
6,9000,True,"(1, 2)",l2,poly,2856.092859,39.370383,-1.235021,0.028726,5,0.607466,3,0.527783,5,0.880646,4


In [23]:
# Exportamos los resultados
SVC_res.to_csv('SVC.csv')

# Testing
add_res(rs_SVC,'SVC')

In [24]:
# Exportamos resultados totales
df_results.to_csv('df_rs.csv')
df_results

,clf,log_loss,accuracy,f1-macro,ROC
0,NB_M,1.458101,0.509036,0.446943,0.828397
1,KNN,2.813324,0.356928,0.365564,0.819434
2,RF,1.851255,0.596386,0.556653,0.863149
3,SVC,1.227287,0.600904,0.531162,0.879490
